In [1]:
import AdventOfCode
import qualified Data.Vector as V

In [2]:
input <- V.fromList . map V.fromList <$> dayLinesString 13

getPos grid (x,y) = (grid V.! y) V.! x

In [3]:
data Turn = LeftTurn | StraightAhead | RightTurn deriving (Eq, Show)
data Heading = U | D | L | R deriving (Eq, Show)
data Cart = Cart
    { cartPos :: (Int, Int)
    , cartHeading :: Heading
    , cartTurn :: Turn
    } deriving (Eq, Show)

parseHeading c = case c of
    '^' -> U
    'v' -> D
    '<' -> L
    '>' -> R

carts = [Cart (x,y) (parseHeading c) LeftTurn | x <- [0..(length (V.head input))-1], y <- [0..(length input)-1], let c = getPos input (x,y), c `elem` "^v<>" ]

In [4]:
underlyingTrack c = case c of
    '^' -> '|'
    'v' -> '|'
    '<' -> '-'
    '>' -> '-'
    _ -> c

grid = V.map (V.map underlyingTrack) input

In [5]:
move cart@(Cart (x,y) heading turn) = case heading of
    U -> updateCart cart { cartPos = (x,y-1) }
    D -> updateCart cart { cartPos = (x,y+1) }
    L -> updateCart cart { cartPos = (x-1,y) }
    R -> updateCart cart { cartPos = (x+1,y) }

updateCart cart@(Cart pos heading turn) gr = case (getPos gr pos) of
    '|' -> cart
    '-' -> cart
    '/' -> case heading of
        U -> cart { cartHeading = R }
        L -> cart { cartHeading = D }
        D -> cart { cartHeading = L }
        R -> cart { cartHeading = U }
    '\\' -> case heading of
        R -> cart { cartHeading = D }
        U -> cart { cartHeading = L }
        D -> cart { cartHeading = R }
        L -> cart { cartHeading = U }
    '+' -> makeTurn cart
  
makeTurn (Cart pos heading turn) = case heading of
    L -> case turn of
        LeftTurn -> Cart pos D StraightAhead
        StraightAhead -> Cart pos L RightTurn
        RightTurn -> Cart pos U LeftTurn
    R -> case turn of
        LeftTurn -> Cart pos U StraightAhead
        StraightAhead -> Cart pos R RightTurn
        RightTurn -> Cart pos D LeftTurn
    U -> case turn of
        LeftTurn -> Cart pos L StraightAhead
        StraightAhead -> Cart pos U RightTurn
        RightTurn -> Cart pos R LeftTurn
    D -> case turn of
        LeftTurn -> Cart pos R StraightAhead
        StraightAhead -> Cart pos D RightTurn
        RightTurn -> Cart pos L LeftTurn

In [6]:
import Data.List
import qualified Data.Set as Set

findDuplicate :: Ord a => [a] -> Set.Set a -> Either a (Set.Set a)
findDuplicate []     set = Right set
findDuplicate (a:as) set = if Set.member a set
    then Left a
    else findDuplicate as (Set.insert a set)

step (c:cs) grid acc = let
    c' = move c grid
    in (cs, (c':acc))

loop cs gr acc = let
    (cs',acc') = step cs gr acc
    summed = cs' ++ acc'
    ps = map cartPos summed
    in case cs' of
        [] -> if length (ordNub ps) < length summed then summed else loop summed gr [] 
        _ -> if length (ordNub ps) < length summed then summed else loop cs' gr acc'
    

flip findDuplicate Set.empty $ map cartPos $ loop carts grid []

Left (83,106)

In [7]:
loop' cs gr acc = let
    (cs',acc') = step cs gr acc
    summed = cs' ++ acc'
    ps = map cartPos summed
    in case summed of
        [x] -> x
        _ -> case findDuplicate ps Set.empty of
            Left c -> let
                cs'' = filter (\cart -> cartPos cart /= c) cs'
                acc'' = filter (\cart -> cartPos cart /= c) acc'
                summed' = cs'' ++ acc''
                in if length summed' == 1 then head summed' else loop' (cs''++acc'') gr []
            Right{} -> loop' summed gr []

In [8]:
loop' carts grid []

Cart {cartPos = (132,26), cartHeading = R, cartTurn = LeftTurn}